# FIAP - Challenge Ingredion – SPRINT 2 - Group 16

## 1 Preprocessamento de Dados

### 1.1 Importando Bibliotecas

In [1]:
import pandas as pd

### 1.2 Carregando Caminhos dos Dados

In [2]:
PRODUTIVITY_PATH = "../data/SIDRA/tabela1613.xlsx"
NDVI_PATH = "../data/SATVEG/satveg_planilha.xlsx"

### 1.3 Carregamento e Processamento de Dados de Produção Agrícola

Esta célula tem como objetivo processar dados brutos de produção agrícola (em toneladas) do município de Manhuaçu (MG), originalmente organizados em tabelas estáticas onde os anos aparecem como colunas e os municípios como linhas, transformando-os em uma série temporal padronizada. A conversão para quilogramas e a reorganização dos dados garantem um formato adequado para análises temporais integradas, além de tornar explícitas informações antes implícitas, como a relação entre colunas sem nomeação específica que representam anos. Essa padronização permite correlacionar os dados históricos de produção com séries de NDVI e outros indicadores agrícolas, o que é essencial para treinar modelos preditivos de produtividade com base em dados consistentes e normalizados.

In [3]:
prod_df = pd.read_excel(
    PRODUTIVITY_PATH,
    sheet_name="Quantidade produzida (Tonela..."
)
prod_df

prod_manhuacu = prod_df[prod_df["Tabela 1613 - Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes"] == "Manhuaçu (MG)"]

year = 1974
rename_dict = {}
for i in range(50):
    col_num = i+1 
    rename_dict[f"Unnamed: {col_num}"] = year
    year += 1

prod_manhuacu = prod_manhuacu.rename(columns=rename_dict)
prod_manhuacu = prod_manhuacu.transpose()
prod_manhuacu = prod_manhuacu.reset_index()
prod_manhuacu = prod_manhuacu.iloc[1:, :]
prod_manhuacu.columns = ["Year", "Production (ton)"]
prod_manhuacu["Production (kg)"] = 1000 * prod_manhuacu["Production (ton)"]
prod_manhuacu = prod_manhuacu.drop(columns=["Production (ton)"])

prod_manhuacu.head()

,Year,Production (kg)
1,1974,4352000
2,1975,1308000
3,1976,2095000
4,1977,5250000
5,1978,5153000


### 1.4 Carregamento e Processamento de Dados de Área Agrícola

Agora vamos processar os dados de área colhida em hectares do município de Manhuaçu (MG), convertendo uma tabela estática (com anos como colunas e municípios como linhas) em uma série temporal estruturada. A partir da planilha original do IBGE, são filtrados apenas os registros do município alvo, renomeadas as colunas não nomeadas (representando anos de 1974 a 2023) para valores explícitos, e reorganizada a estrutura dos dados por meio de transposição e ajustes de índice. O resultado final é um DataFrame limpo com as colunas "Year" e "Area (ha)", contendo a área cultivada anual em hectares, pronto para ser cruzado com os dados de produção e permitir o cálculo preciso da produtividade agrícola (kg/ha) ao longo do tempo.

In [4]:
area_df = pd.read_excel(
    PRODUTIVITY_PATH,
    sheet_name="Área colhida (Hectares)"
)

area_manhuacu = area_df[area_df["Tabela 1613 - Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes"] == "Manhuaçu (MG)"]

year = 1974
rename_dict = {}
for i in range(50):
    col_num = i+1 
    rename_dict[f"Unnamed: {col_num}"] = year
    year += 1

area_manhuacu = area_manhuacu.rename(columns=rename_dict)
area_manhuacu = area_manhuacu.transpose()
area_manhuacu = area_manhuacu.reset_index()
area_manhuacu = area_manhuacu.iloc[1:, :]
area_manhuacu.columns = ["Year", "Area (ha)"]

area_manhuacu.head()

,Year,Area (ha)
1,1974,2176
2,1975,2180
3,1976,2538
4,1977,2625
5,1978,2931


### 1.5 Consolidação de Dados Agrícolas

Agora vamos criar um dataset unificado contendo produção agrícola (kg) e área cultivada (ha) por ano para Manhuaçu (MG), permitindo o cálculo da produtividade (kg/ha).

In [5]:
manhuacu_df = pd.merge(
    how="inner",
    left=prod_manhuacu,
    right=area_manhuacu,
    on="Year",
)
manhuacu_df.head()

,Year,Production (kg),Area (ha)
0,1974,4352000,2176
1,1975,1308000,2180
2,1976,2095000,2538
3,1977,5250000,2625
4,1978,5153000,2931


### 1.6 Cálculo de Produtividade Agrícola

Calcular a produtividade agrícola anual (kg/hectare) de Manhuaçu (MG) a partir dos dados consolidados de produção e área cultivada.

In [6]:
manhuacu_df["Productivity (kg/ha)"] = manhuacu_df["Production (kg)"] / manhuacu_df["Area (ha)"]
manhuacu_df.head()

,Year,Production (kg),Area (ha),Productivity (kg/ha)
0,1974,4352000,2176,2000.0
1,1975,1308000,2180,600.0
2,1976,2095000,2538,825.453113
3,1977,5250000,2625,2000.0
4,1978,5153000,2931,1758.103037


### 1.7 Exportando os Dados Processados

Persistir o dataset consolidado de produtividade agrícola em formato CSV para uso em etapas posteriores do projeto.

In [7]:
manhuacu_df.to_csv(
    "../data/PROCESSED/manhuacu.csv",
    index=False,
)

### 1.8 Processamento de Dados de NDVI

Estruturar a série temporal de NDVI (Índice de Vegetação) para integração com os dados agrícolas, garantindo compatibilidade temporal e formato adequado para análise.

In [8]:
NDVI_df = pd.read_excel(
    NDVI_PATH,
    sheet_name="NDVI",
    skiprows=3
)

NDVI_df = NDVI_df.iloc[:, :2]
NDVI_df["Data"] = pd.to_datetime(NDVI_df["Data"], format="%d/%m/%Y")
NDVI_df = NDVI_df.sort_values(by="Data", ascending=True)
NDVI_df["Year"] = NDVI_df["Data"].dt.year

NDVI_df.to_csv(
    "../data/PROCESSED/ndvi.csv",
    index=False,
)

NDVI_df.head()

/home/luis/GitHub/fiap/ec-ingredion-2/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Data,NDVI,Year
0,2000-02-18,0.4925,2000
1,2000-03-05,0.7898,2000
2,2000-03-21,0.8125,2000
3,2000-04-06,0.7585,2000
4,2000-04-22,0.7437,2000
